#### **Install necessary Libraries**

In [ ]:
!pip install torch
!pip install torch-geometric
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-<YOUR-TORCH-VERSION>+cpu.html
!pip install torch torchvision torchaudio torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 20.6 MB/s eta 0:00:00
/bin/bash: line 1: YOUR-TORCH-VERSION: No such file or directory


#### **Import the required Libraries**

In [ ]:
import torch                                      # The main PyTorch library for tensor computation.
import torch.nn as nn                             # Provides classes and functions for building neural networks.
import torch.optim as optim                       # Contains various optimization algorithms for training neural networks.
from torch_geometric.nn import GCNConv            # A Graph Convolutional Layer from PyTorch Geometric.
from torch_geometric.nn import GATConv            # A Graph Attention Network (GAT) Convolutional Layer from PyTorch Geometric.
from torch_geometric.nn import SAGEConv           # A GraphSAGE Convolutional Layer from PyTorch Geometric.
from torch_geometric.nn import TransformerConv    # A Transformer Convolutional Layer from PyTorch Geometric.
import torch.nn.functional as F                   # Contains various functions for building neural networks (e.g., activation functions).
from torch_geometric.data import Data             # A class for graph data in PyTorch Geometric.
from torch.optim import Adam                      # Adam optimization algorithm for training neural networks.
from torch.nn.functional import cross_entropy     # Cross-entropy loss function for classification tasks.
from torch_geometric.loader import NeighborLoader # Loads graph data with neighbor sampling for efficient training.
from torch_geometric.loader import DataLoader     # A DataLoader for graph data in PyTorch Geometric.
import networkx as nx                             # Library for graph and network analysis.
from torch_geometric.utils import from_networkx   # Converts a NetworkX graph to PyTorch Geometric format.
from torch_geometric.utils import subgraph        # Extracts a subgraph from a larger graph.
from sklearn.preprocessing import StandardScaler  # Standardizes features by removing the mean and scaling to unit variance.
from sklearn.model_selection import train_test_split # Splits datasets into training and testing subsets.
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score # Metrics for evaluating models.
from tqdm import tqdm                             # For progress tracking in loops.
import networkx as nx                             # Library for creating and analyzing graph data structures.
import pickle                                     # For saving and loading Python objects (e.g., models, data).
import os                                         # For file and directory operations.
import itertools                                  # For working with iterators and combinations.
from itertools import product                     # For generating the Cartesian product of input iterables.
# Import python packages
import pandas as pd                               # For data manipulation and analysis.
import numpy as np                                # For numerical computations.
import seaborn as sns                             # For data visualization.
import matplotlib.pyplot as plt                   # For creating visualizations.
# To ignore warnings
import warnings                                   # Handles Python warnings.
warnings.filterwarnings("ignore")                # Suppresses all warnings.


#### **Load Node data and convert to pickle**

In [ ]:
# Load the resampled data from CSV
graph_data = pd.read_csv('/content/drive/MyDrive/GraphFeatures/graph_data.csv')

# Assuming node_data is created as a copy of graph_data
node_data = graph_data.copy()

# Define the file path in Google Drive
node_data_file_path = '/content/drive/MyDrive/GraphFeatures/node_data.pkl'

# Save node_data as a pickle file
with open(node_data_file_path, 'wb') as f:
    pickle.dump(node_data, f)

print(f"node_data saved successfully to {node_data_file_path}!")


node_data saved successfully to /content/drive/MyDrive/GraphFeatures/node_data.pkl!


#### **Load Node-Centric Graph and Node_Data**

In [ ]:
# Load the saved node-centric graph
with open('/content/drive/MyDrive/GraphFeatures/NodeCentricGraph.pkl', 'rb') as f:
    G = pickle.load(f)

# Define the file path in Google Drive
node_data_file_path = '/content/drive/MyDrive/GraphFeatures/node_data.pkl'

# Load the node_data from the pickle file
with open(node_data_file_path, 'rb') as f:
    node_data = pickle.load(f)

#### **Convert the Node-Centric Graph to PyTorch Geometric Format**

In [ ]:
# Convert NetworkX graph to PyTorch Geometric data
data = from_networkx(G)

# Extract target labels (isFraud) and ensure data type consistency
data.y = torch.tensor(
    [node_data.loc[node_data['TransactionID'] == node, 'isFraud'].values[0]
     for node in G.nodes],
    dtype=torch.long  # Ensure labels are integers for classification
)

# Extract node features and ensure consistent dtype
data.x = torch.tensor(
    [list(G.nodes[node].values()) for node in G.nodes],
    dtype=torch.float  # Features should be float for GNN layers
)

#### **Scale Node Features**

In [ ]:
# Scale node features and maintain consistent dtype
scaler = StandardScaler()
data.x = torch.tensor(
    scaler.fit_transform(data.x.numpy()),
    dtype=torch.float  # Ensure consistent dtype after scaling
)

#### **Train Test Split**

In [ ]:
# Split the data into train and temporary (validation + test) sets
train_indices, temp_indices = train_test_split(
    range(data.num_nodes),
    test_size=0.4,  # 40% will be split further into validation and test sets
    random_state=42
)

# Further split the temporary set into validation and test sets
val_indices, test_indices = train_test_split(
    temp_indices,
    test_size=0.5,  # Half of the remaining 40% (20% of total) for test
    random_state=42
)

# Create boolean masks for train, validation, and test sets
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

data.train_mask[train_indices] = True
data.val_mask[val_indices] = True
data.test_mask[test_indices] = True

# Print the number of nodes in each split
print(f"Train nodes: {data.train_mask.sum().item()}")
print(f"Validation nodes: {data.val_mask.sum().item()}")
print(f"Test nodes: {data.test_mask.sum().item()}")


Train nodes: 59938
Validation nodes: 19980
Test nodes: 19980


#### **Node-Centric Graph with Graph Convolution Network (GCN) Model**

##### **Define the Model**

In [ ]:
class GCNModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


##### **Initialize and Train the Model**

In [ ]:
# Model parameters
input_dim = data.x.size(1)  # Feature dimension
hidden_dim = 32
output_dim = len(torch.unique(data.y))  # Number of classes

# Initialize the model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCNModel(input_dim, hidden_dim, output_dim).to(device)
data = data.to(device)
optimizer = Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 50
best_val_loss = float('inf')
patience = 5
early_stop_counter = 0

for epoch in range(num_epochs):
    # Training phase
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    train_loss = criterion(out[data.train_mask], data.y[data.train_mask])
    train_loss.backward()
    optimizer.step()

    # Validation phase
    model.eval()
    with torch.no_grad():
        val_out = model(data.x, data.edge_index)
        val_loss = criterion(val_out[data.val_mask], data.y[data.val_mask])

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break


Epoch 1/50, Train Loss: 0.7191, Val Loss: 0.6710
Epoch 2/50, Train Loss: 0.6693, Val Loss: 0.6705
Epoch 3/50, Train Loss: 0.6698, Val Loss: 0.6623
Epoch 4/50, Train Loss: 0.6624, Val Loss: 0.6612
Epoch 5/50, Train Loss: 0.6617, Val Loss: 0.6570
Epoch 6/50, Train Loss: 0.6574, Val Loss: 0.6506
Epoch 7/50, Train Loss: 0.6506, Val Loss: 0.6516
Epoch 8/50, Train Loss: 0.6513, Val Loss: 0.6553
Epoch 9/50, Train Loss: 0.6548, Val Loss: 0.6552
Epoch 10/50, Train Loss: 0.6547, Val Loss: 0.6521
Epoch 11/50, Train Loss: 0.6517, Val Loss: 0.6495
Epoch 12/50, Train Loss: 0.6492, Val Loss: 0.6485
Epoch 13/50, Train Loss: 0.6484, Val Loss: 0.6476
Epoch 14/50, Train Loss: 0.6475, Val Loss: 0.6469
Epoch 15/50, Train Loss: 0.6468, Val Loss: 0.6476
Epoch 16/50, Train Loss: 0.6475, Val Loss: 0.6492
Epoch 17/50, Train Loss: 0.6489, Val Loss: 0.6496
Epoch 18/50, Train Loss: 0.6493, Val Loss: 0.6483
Epoch 19/50, Train Loss: 0.6480, Val Loss: 0.6466
Epoch 20/50, Train Loss: 0.6464, Val Loss: 0.6458
Epoch 21/

##### **Evaluate the GCN Model**

In [ ]:
# Evaluation function
def evaluate_model(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        preds = out.argmax(dim=1)  # Predicted labels

        # Test set metrics
        y_true = data.y[data.test_mask].cpu().numpy()
        y_pred = preds[data.test_mask].cpu().numpy()

        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        auc_roc = roc_auc_score(y_true, out[data.test_mask][:, 1].cpu().numpy())

        print("Test Metrics:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-Score: {f1:.4f}")
        print(f"AUC-ROC: {auc_roc:.4f}")

# Evaluate the trained model
evaluate_model(model, data)

Test Metrics:
Accuracy: 0.6435
Precision: 0.7013
Recall: 0.5038
F1-Score: 0.5864
AUC-ROC: 0.6519


#### **Node-Centric Graph with GAT Model**

##### **Define GAT Model**

In [ ]:
class GATModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads):
        super(GATModel, self).__init__()
        self.conv1 = GATConv(input_dim, hidden_dim, heads=num_heads, dropout=0.6)
        self.conv2 = GATConv(hidden_dim * num_heads, output_dim, heads=1, dropout=0.6)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)  # Activation function
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


##### **Initialize and Train GAT Model**

In [ ]:
# Model parameters
input_dim = data.x.size(1)  # Feature dimension
hidden_dim = 32
output_dim = len(torch.unique(data.y))  # Number of classes
num_heads = 4

# Initialize the model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GATModel(input_dim, hidden_dim, output_dim, num_heads).to(device)
data = data.to(device)
optimizer = Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 50
best_val_loss = float('inf')
patience = 5
early_stop_counter = 0

for epoch in range(num_epochs):
    # Training phase
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    train_loss = criterion(out[data.train_mask], data.y[data.train_mask])
    train_loss.backward()
    optimizer.step()

    # Validation phase
    model.eval()
    with torch.no_grad():
        val_out = model(data.x, data.edge_index)
        val_loss = criterion(val_out[data.val_mask], data.y[data.val_mask])

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break


Epoch 1/50, Train Loss: 21.2482, Val Loss: 2.9551
Epoch 2/50, Train Loss: 10.8220, Val Loss: 3.1767
Epoch 3/50, Train Loss: 3.4737, Val Loss: 2.8836
Epoch 4/50, Train Loss: 1.9550, Val Loss: 1.4459
Epoch 5/50, Train Loss: 1.1774, Val Loss: 1.1910
Epoch 6/50, Train Loss: 1.1336, Val Loss: 1.0578
Epoch 7/50, Train Loss: 1.0314, Val Loss: 1.0151
Epoch 8/50, Train Loss: 0.9815, Val Loss: 0.9731
Epoch 9/50, Train Loss: 0.8813, Val Loss: 0.9434
Epoch 10/50, Train Loss: 0.8761, Val Loss: 0.9285
Epoch 11/50, Train Loss: 0.8369, Val Loss: 0.9321
Epoch 12/50, Train Loss: 0.8205, Val Loss: 0.9506
Epoch 13/50, Train Loss: 0.7906, Val Loss: 0.9869
Epoch 14/50, Train Loss: 0.8233, Val Loss: 1.0341
Epoch 15/50, Train Loss: 0.8408, Val Loss: 1.0443
Early stopping at epoch 15


##### **Evaluate the Model**

In [ ]:
# Evaluation function
def evaluate_model(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        preds = out.argmax(dim=1)  # Predicted labels

        # Test set metrics
        y_true = data.y[data.test_mask].cpu().numpy()
        y_pred = preds[data.test_mask].cpu().numpy()

        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        auc_roc = roc_auc_score(y_true, out[data.test_mask][:, 1].cpu().numpy())

        print("Test Metrics:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-Score: {f1:.4f}")
        print(f"AUC-ROC: {auc_roc:.4f}")

# Evaluate the trained model
evaluate_model(model, data)


Test Metrics:
Accuracy: 0.6026
Precision: 0.6882
Recall: 0.3797
F1-Score: 0.4894
AUC-ROC: 0.6000


#### **Node-Centric Graph with GraphSAGE Model**

##### **Define the Model**

In [ ]:
class GraphSAGEModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphSAGEModel, self).__init__()
        self.conv1 = SAGEConv(input_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


##### **Initialize and Train the Model**

In [ ]:
# Model parameters
input_dim = data.x.size(1)  # Feature dimension
hidden_dim = 32
output_dim = len(torch.unique(data.y))  # Number of classes

# Initialize the model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphSAGEModel(input_dim, hidden_dim, output_dim).to(device)
data = data.to(device)
optimizer = Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 50
best_val_loss = float('inf')
patience = 5
early_stop_counter = 0

for epoch in range(num_epochs):
    # Training phase
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    train_loss = criterion(out[data.train_mask], data.y[data.train_mask])
    train_loss.backward()
    optimizer.step()

    # Validation phase
    model.eval()
    with torch.no_grad():
        val_out = model(data.x, data.edge_index)
        val_loss = criterion(val_out[data.val_mask], data.y[data.val_mask])

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break


Epoch 1/50, Train Loss: 0.7330, Val Loss: 0.6581
Epoch 2/50, Train Loss: 0.6653, Val Loss: 0.6342
Epoch 3/50, Train Loss: 0.6423, Val Loss: 0.6233
Epoch 4/50, Train Loss: 0.6324, Val Loss: 0.6121
Epoch 5/50, Train Loss: 0.6214, Val Loss: 0.6001
Epoch 6/50, Train Loss: 0.6103, Val Loss: 0.5871
Epoch 7/50, Train Loss: 0.5972, Val Loss: 0.5751
Epoch 8/50, Train Loss: 0.5850, Val Loss: 0.5660
Epoch 9/50, Train Loss: 0.5763, Val Loss: 0.5598
Epoch 10/50, Train Loss: 0.5709, Val Loss: 0.5551
Epoch 11/50, Train Loss: 0.5653, Val Loss: 0.5503
Epoch 12/50, Train Loss: 0.5626, Val Loss: 0.5449
Epoch 13/50, Train Loss: 0.5583, Val Loss: 0.5397
Epoch 14/50, Train Loss: 0.5507, Val Loss: 0.5353
Epoch 15/50, Train Loss: 0.5490, Val Loss: 0.5317
Epoch 16/50, Train Loss: 0.5454, Val Loss: 0.5286
Epoch 17/50, Train Loss: 0.5420, Val Loss: 0.5257
Epoch 18/50, Train Loss: 0.5401, Val Loss: 0.5231
Epoch 19/50, Train Loss: 0.5384, Val Loss: 0.5207
Epoch 20/50, Train Loss: 0.5340, Val Loss: 0.5183
Epoch 21/

##### **Evaluate the Model**

In [ ]:
# Evaluation function
def evaluate_model(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        preds = out.argmax(dim=1)  # Predicted labels

        # Test set metrics
        y_true = data.y[data.test_mask].cpu().numpy()
        y_pred = preds[data.test_mask].cpu().numpy()

        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        auc_roc = roc_auc_score(y_true, out[data.test_mask][:, 1].cpu().numpy())

        print("Test Metrics:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-Score: {f1:.4f}")
        print(f"AUC-ROC: {auc_roc:.4f}")

# Evaluate the trained model
evaluate_model(model, data)


Test Metrics:
Accuracy: 0.7788
Precision: 0.7691
Recall: 0.7988
F1-Score: 0.7837
AUC-ROC: 0.8550


#### **Node-Centric Graph with Graphomer Transformer Model**

#### **Define Model**

In [ ]:
class Graphormer(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads, dropout=0.1):
        super(Graphormer, self).__init__()
        self.conv1 = TransformerConv(input_dim, hidden_dim, heads=num_heads, dropout=dropout)
        self.conv2 = TransformerConv(hidden_dim * num_heads, output_dim, heads=1, dropout=dropout)
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


##### **Initialize and Train the Graphomer Model**

In [ ]:
# Model parameters
input_dim = data.x.size(1)  # Number of input features
hidden_dim = 64  # Hidden layer dimension
output_dim = len(torch.unique(data.y))  # Number of classes
num_heads = 4  # Number of attention heads

# Initialize the model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Graphormer(input_dim, hidden_dim, output_dim, num_heads).to(device)
data = data.to(device)
optimizer = Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 50
best_val_loss = float('inf')
patience = 5
stopping_counter = 0

for epoch in range(num_epochs):
    # Training phase
    model.train()
    optimizer.zero_grad()

    out = model(data.x, data.edge_index)
    train_loss = criterion(out[data.train_mask], data.y[data.train_mask])
    train_loss.backward()
    optimizer.step()

    # Validation phase
    model.eval()
    with torch.no_grad():
        val_out = model(data.x, data.edge_index)
        val_loss = criterion(val_out[data.val_mask], data.y[data.val_mask])

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        stopping_counter = 0
    else:
        stopping_counter += 1
        if stopping_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

Epoch 1/50, Train Loss: 0.8653, Val Loss: 0.8049
Epoch 2/50, Train Loss: 0.7990, Val Loss: 0.6858
Epoch 3/50, Train Loss: 0.6911, Val Loss: 0.6013
Epoch 4/50, Train Loss: 0.6014, Val Loss: 0.5715
Epoch 5/50, Train Loss: 0.5696, Val Loss: 0.5594
Epoch 6/50, Train Loss: 0.5594, Val Loss: 0.5528
Epoch 7/50, Train Loss: 0.5551, Val Loss: 0.5436
Epoch 8/50, Train Loss: 0.5465, Val Loss: 0.5310
Epoch 9/50, Train Loss: 0.5341, Val Loss: 0.5186
Epoch 10/50, Train Loss: 0.5202, Val Loss: 0.5191
Epoch 11/50, Train Loss: 0.5194, Val Loss: 0.5165
Epoch 12/50, Train Loss: 0.5163, Val Loss: 0.5112
Epoch 13/50, Train Loss: 0.5108, Val Loss: 0.5061
Epoch 14/50, Train Loss: 0.5058, Val Loss: 0.5001
Epoch 15/50, Train Loss: 0.4991, Val Loss: 0.4973
Epoch 16/50, Train Loss: 0.4964, Val Loss: 0.4909
Epoch 17/50, Train Loss: 0.4896, Val Loss: 0.4883
Epoch 18/50, Train Loss: 0.4872, Val Loss: 0.4877
Epoch 19/50, Train Loss: 0.4852, Val Loss: 0.4831
Epoch 20/50, Train Loss: 0.4792, Val Loss: 0.4773
Epoch 21/

#### **Evaluate the Model**

In [ ]:
# Evaluation function
def evaluate_model(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        preds = out.argmax(dim=1)  # Predicted labels

        # Extract test set metrics
        y_true = data.y[data.test_mask].cpu().numpy()
        y_pred = preds[data.test_mask].cpu().numpy()

        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        auc_roc = roc_auc_score(y_true, out[data.test_mask][:, 1].cpu().numpy())

        print("Test Set Metrics:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-Score: {f1:.4f}")
        print(f"AUC-ROC: {auc_roc:.4f}")

# Evaluate the trained model
evaluate_model(model, data)


Test Set Metrics:
Accuracy: 0.8240
Precision: 0.8264
Recall: 0.8216
F1-Score: 0.8240
AUC-ROC: 0.9014
